Credits go to:

Boudina, Mouad. (2021). Numerical simulation data of a two-dimensional flow around a fixed circular cylinder [Data set]. Zenodo. https://doi.org/10.5281/zenodo.5039610

In [ ]:
import requests
import pandas as pd
import time
import numpy as np

### Download DNS Data from URL

In [ ]:
URL = "https://zenodo.org/record/5039610/files/fixed_cylinder_atRe100?download=1"
response = requests.get(URL)
open("fixed_cylinder_atRe100", "wb").write(response.content)

### Auxiliary Script to Process Downloaded Data File

In [ ]:
def floatIt(l):
    return np.array([float(e) for e in l])

def intIt(l):
    return np.array([int(e) for e in l])

def read_flow(infile):
    f = open(infile, 'r')
    t1 = time.time()
    print('Reading flow...')

    Re, Ur = floatIt(f.readline().strip().split())
    f.readline() # blank line
    Nt, N_nodes = intIt(f.readline().strip().split())
    f.readline()
    
    times = []
    nodes_X, nodes_Y = [], []
    Us, Vs, ps = [], [], []

    for n in range(Nt):
        tn = float(f.readline().strip())
        times.append(tn)

        print('%.3f' % tn)

        tmp_nodes_X, tmp_nodes_Y = [], []
        tmp_Us, tmp_Vs, tmp_ps = [], [], []

        for k in range(N_nodes):
            x, y, U, V, p = floatIt(f.readline().strip().split())

            tmp_nodes_X.append(x)
            tmp_nodes_Y.append(y)

            tmp_Us.append(U)
            tmp_Vs.append(V)
            tmp_ps.append(p)

        nodes_X.append(tmp_nodes_X)
        nodes_Y.append(tmp_nodes_Y)

        Us.append(tmp_Us)
        Vs.append(tmp_Vs)
        ps.append(tmp_ps)

    cpu_time = time.time() - t1
    print('Done!')
    print('CPU_TIME = %f seconds' % cpu_time)

    f.close()

    return Re, Ur, np.array(times), \
           np.array(nodes_X), np.array(nodes_Y), \
           np.array(Us), np.array(Vs), np.array(ps)

### Process File

In [ ]:
Re, Ur, times, nodes_X, nodes_Y, Us, Vs, ps = read_flow('fixed_cylinder_atRe100')
N_nodes = nodes_X.shape[1]
t_min = min(times)
X = nodes_X[1]
Y = nodes_Y[1]

### Data to (Pandas) CSV File

In [ ]:
dataframe = []
for i, t in enumerate(times):
    for node in range(N_nodes):
        row = [t-t_min, nodes_X[i][node], nodes_Y[i][node], Us[i][node], Vs[i][node], ps[i][node]]
        dataframe.append(row)
        
dataframe = pd.DataFrame(dataframe, columns=['t', 'x', 'y', 'u', 'v', 'p'])
dataframe.to_csv('navier_stokes.csv', index=False)